In [1]:
import os

In [2]:
%pwd

'd:\\Project\\P\\Mlops-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Project\\P\\Mlops-Project'

In [6]:
import tensorflow as tf

In [5]:
import pandas as pd
data = pd.read_csv("artifacts/data_transformation/train.csv")
data.head()

,ImageID,XMin,XMax,YMin,YMax
0,c640a3584d1e4d71,0.514749,0.641593,0.800885,1.000000
1,d819685832a0e19b,0.429204,0.659292,0.449115,0.597345
2,c705061238b1fe30,0.317187,1.000000,0.325000,0.997917
3,18245b3e96479da4,0.830389,0.902827,0.385609,0.645756
4,02717d30304f4849,0.014062,0.045313,0.439583,0.537500


In [55]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.backend import std, mean
from tensorflow.math import reduce_std, reduce_mean

def _normalize(image_path, bbox):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    # img = tf.image.resize(img, [460,])  # Resize as needed

    bbox = tf.cast(bbox, dtype=tf.float32)
    return img, bbox


train_data = pd.read_csv("artifacts/data_transformation/train.csv")  # assuming this is the correct path for train.csv
train_image_paths = train_data["ImageID"].apply(lambda x: "artifacts/data_transformation/train/" + x + ".jpg").tolist()
train_boxes = train_data[["XMin", "YMin", "XMax", "YMax"]].values.tolist()


valid_data = pd.read_csv("artifacts/data_transformation/test.csv")
valid_image_paths = valid_data["ImageID"].apply(lambda x: "artifacts/data_transformation/test/" + x + ".jpg").tolist()
valid_boxes = valid_data[["XMin", "YMin", "XMax", "YMax"]].values.tolist()

train_dataset = tf.data.Dataset.from_tensor_slices((train_image_paths, train_boxes))
train_dataset = train_dataset.map(_normalize, num_parallel_calls=tf.data.AUTOTUNE).batch(32)

valid_dataset = tf.data.Dataset.from_tensor_slices((valid_image_paths, valid_boxes))
valid_dataset = valid_dataset.map(_normalize, num_parallel_calls=tf.data.AUTOTUNE).batch(32)

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelTrainerConfig:
    root_dir: Path
    all_data: Path
    image_train_data: Path
    image_test_data: Path
    model_name: str
    

In [5]:
from MLOps_project.constant import *
from MLOps_project.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            all_data = config.all_data,
            image_train_data = config.image_train_data,
            image_test_data = config.image_test_data,
            model_name = config.model_name
        )

        return model_trainer_config

In [ ]:
from ultralytics import YOLO

In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def create_yaml(self, path, train_folder, val_folder):
        yaml_content = f"""\
                        # Path
                        path: {path}
                        train: {train_folder}
                        val: {val_folder}

                        # Classes
                        nc: 1
                        names: ['person']
                        """
        with open('dataset.yaml', 'w') as file:
            file.write(yaml_content)

    def train(self):
        self.create_yaml(path=self.config.root_dir, train_folder=self.config.image_train_data, val_folder=self.config.image_test_data)
        model = YOLO('yolov5n.pt')
        model.train(data="dataset.yaml", epochs=30, batch=8)
        


In [ ]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e